In [2]:
import pandas
import numpy
import pygsheets

In [3]:
pandas.__version__

'1.5.0'

# credentials info

In [4]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1v38EY_kFFsT17cPqz39j21zRYWeQSSxuDmz565MD2GU') # copy for 2022 report

# sheet 1 (old)

In [5]:
#spreadsheet1 = gc.open_by_key('1MrghwBeCz8Tzgua7CWGg_KoXKVZsV7r0kHMYHYqnNTg') # Aug 18 version
spreadsheet1 = gc.open_by_key('1-dJ_vK3G9pbfTuriMphfRD65ybC99qv3VpjWzMicCzQ') # Aug 19 version, official database

#spreadsheet[1] "Gas Pipelines" tab is the second index
terms_df_orig1 = spreadsheet1.worksheet('title', 'Terminals').get_as_df()

In [6]:
# replace all -- with nans
terms_df_orig1.replace('--', numpy.nan, inplace=True)
# remove oil export terminals
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Fuel']=='LNG']
# remove anything without a wiki page
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Wiki']!='']
# remove N/A statuses
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Status']!='']

terms_df_orig1.set_index('ComboID', inplace=True)

# sheet 2 (new)

In [7]:
#spreadsheet2 = gc.open_by_key('1tcS6Wd-Wp-LTDpLzFgJY_RSNDnbyubW3J_9HKIAys4A') # present-day data set
spreadsheet2 = gc.open_by_key('1pO2ltCBrzcQuJlem9GwlWQm3YwjDOgjPGx166_OeZmY') # as of 4 Nov 2022
#spreadsheet2 = gc.open_by_key('1gAo21abcKUW9r60qWKTAslbMe8vnAZ5voVEFdJ9-aAo')
terms_df_orig2 = spreadsheet2.worksheet('title', 'Terminals').get_as_df(start='A2')

In [8]:
# replace all -- with nans
terms_df_orig2.replace('--', numpy.nan, inplace=True)
# remove oil export terminals
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Fuel']=='LNG']
# remove anything without a wiki page
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Wiki']!='']
# remove N/A statuses
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Status']!='']

In [9]:
#terms_df_orig2['TerminalName'] = terms_df_orig2.TerminalName.astype(str)+' '+terms_df_orig2.UnitName.astype(str)
terms_df_orig2.set_index('ComboID', inplace=True)

## get shared columns

In [108]:
keep_these_columns = list(set(terms_df_orig2.columns).intersection(set(terms_df_orig1.columns)))

## now get shared rows

In [109]:
if 'ComboID' in terms_df_orig2.columns:
    terms_df_orig2.set_index('ComboID', inplace=True)
if 'ComboID' in terms_df_orig1.columns:
    terms_df_orig1.set_index('ComboID', inplace=True)

In [110]:
remove_these_indices = set(terms_df_orig2.index).symmetric_difference(set(terms_df_orig1.index))

In [111]:
new_additions = list(set(terms_df_orig2.index)-set(terms_df_orig1.index))
deleted_since_feb = list(set(terms_df_orig1.index)-set(terms_df_orig2.index))

In [112]:
keep_these_indices = set(terms_df_orig2.index).intersection(set(terms_df_orig1.index))

## if there are diff ComboIDs, pull out only the common ones, THEN compare rest

In [113]:
common_df1 = terms_df_orig1.loc[terms_df_orig1.index.isin(keep_these_indices),keep_these_columns].sort_index()
common_df2 = terms_df_orig2.loc[terms_df_orig2.index.isin(keep_these_indices),keep_these_columns].sort_index()

common_df1 = common_df1.loc[common_df1.Region=='Europe']
common_df2 = common_df2.loc[common_df2.Region=='Europe']

In [114]:
#index_list = common_df2.compare(common_df1, result_names=('after','before')).index
common_df2.compare(common_df1, result_names=('after','before')).replace(numpy.nan,'')

CapacityInMtpa        CancelledYear        PowerPlantsSupplied         \
                 after before         after before               after before   
ComboID                                                                         
T042400                                2012                                     
T042600                                                                         
T042601                                                                         
T042602                                                                         
T042800                                                                         
...                ...    ...           ...    ...                 ...    ...   
T083100                                                                         
T085500           1.91    2.4                                                   
T085600                                                                         
T092400            3.5                                                          
T099200                                                                         

          Latitude            Opposition         ... ShelvedYear         \
             after     before      after before  ...       after before   
ComboID                                          ...                      
T042400                                          ...        2010          
T042600                                          ...                      
T042601                                          ...                      
T042602                                          ...                      
T042800                                          ...                      
...            ...        ...        ...    ...  ...         ...    ...   
T083100                                          ...                      
T085500  51.264274  51.088313                    ...                      
T085600                                          ...                      
T092400                                          ...                      
T099200                                          ...                      

        Researcher        StopYear        ConstructionYear         PCI4         
             after before    after before            after before after before  
ComboID                                                                         
T042400                                                                         
T042600                                               1982                      
T042601         GA                                                              
T042602         GA                                                              
T042800                                                                         
...            ...    ...      ...    ...              ...    ...   ...    ...  
T083100                                                                         
T085500         GC                                                              
T085600         GC                                                              
T092400         GC                                                              
T099200                                                                         

[112 rows x 74 columns]

In [115]:
diff_df = common_df2.compare(common_df1, result_names=('after','before')).replace(numpy.nan,'')

In [116]:
diff_df[('Region','')] = terms_df_orig2.loc[diff_df.index,'Region']
diff_df[('TerminalName','')] = terms_df_orig2.loc[diff_df.index, 'TerminalName']
diff_df[('Wiki','')] = terms_df_orig2.loc[diff_df.index, 'Wiki']

In [117]:
diff_df.to_excel('differences_before_after_russian_invasion.xlsx')#, index=False)

In [118]:
added_df = terms_df_orig2.loc[new_additions]

In [119]:
added_df['UnitName'] = terms_df_orig2.loc[added_df.index,'UnitName']

In [120]:
removed_df = terms_df_orig1.loc[deleted_since_feb]

In [121]:
added_df.to_excel('added_after_russian_invasion.xlsx')
removed_df.to_excel('removed_after_russian_invasion.xlsx')

## how did change globally, Europe, Germany?    

### globally

In [122]:
# num of distinct projects
new_comboids = list(set(terms_df_orig2.index)-set(terms_df_orig1.index))
new_projectids = list(set(terms_df_orig2.ProjectID)-set(terms_df_orig1.ProjectID))

print(new_comboids.__len__())
print(new_projectids.__len__())

128
5


# orig1

In [123]:
print(terms_df_orig1.shape)
print(terms_df_orig1.TerminalID.unique().size)
print(terms_df_orig1.index.unique().size)

(1092, 61)
696
1092


In [124]:
print(terms_df_orig1.loc[terms_df_orig1.Region=='Europe'].shape)
print(terms_df_orig1.loc[terms_df_orig1.Region=='Europe'].TerminalID.unique().size)
print(terms_df_orig1.loc[terms_df_orig1.Region=='Europe'].index.unique().size)

(113, 61)
90
113


In [125]:
print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].shape)
print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].TerminalID.unique().size)
print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].index.unique().size)

(33, 61)
28
33


In [126]:
print(terms_df_orig1.loc[terms_df_orig1.Country=='Germany'].shape)
print(terms_df_orig1.loc[terms_df_orig1.Country=='Germany'].TerminalID.unique().size)
print(terms_df_orig1.loc[terms_df_orig1.Country=='Germany'].index.unique().size)

(4, 61)
4
4


In [127]:
print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].shape)
print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].TerminalID.unique().size)
print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].index.unique().size)

(2, 61)
2
2


# orig2

In [128]:
print(terms_df_orig2.shape)
print(terms_df_orig2.TerminalID.unique().size)
print(terms_df_orig2.index.unique().size)

(1182, 74)
735
1182


In [129]:
print(terms_df_orig2.loc[terms_df_orig2.Region=='Europe'].shape)
print(terms_df_orig2.loc[terms_df_orig2.Region=='Europe'].TerminalID.unique().size)
print(terms_df_orig2.loc[terms_df_orig2.Region=='Europe'].index.unique().size)

(131, 74)
105
131


In [130]:
print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].shape)
print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].TerminalID.unique().size)
print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].index.unique().size)

(46, 74)
41
46


In [131]:
print(terms_df_orig2.loc[terms_df_orig2.Country=='Germany'].shape)
print(terms_df_orig2.loc[terms_df_orig2.Country=='Germany'].TerminalID.unique().size)
print(terms_df_orig2.loc[terms_df_orig2.Country=='Germany'].index.unique().size)

(9, 74)
9
9


In [132]:
print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].shape)
print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].TerminalID.unique().size)
print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].index.unique().size)

(8, 74)
8
8


# cost differences

In [133]:
region_list = sorted(terms_df_orig2.Region.unique())
status_list = ['Proposed', 'Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']
country_list = sorted(list(set(terms_df_orig2['Country'])))

In [134]:
terms_df_floating = terms_df_orig2.loc[terms_df_orig2.Floating=='yes']
terms_df_onshore = terms_df_orig2.loc[terms_df_orig2.Floating=='']

#terms_df_floating = terms_df_floating.loc[~terms_df_floating.ComboID.isin(outlier_comboids)]
#terms_df_onshore = terms_df_onshore.loc[~terms_df_onshore.ComboID.isin(outlier_comboids)]
terms_df_floating = terms_df_floating.loc[terms_df_floating.CostUSDPerMtpa<1e9]
terms_df_onshore = terms_df_onshore.loc[terms_df_onshore.CostUSDPerMtpa<1e9]

### delete repeated money values

In [135]:
terms_df_floating_noexpansions = terms_df_floating.loc[~terms_df_floating['UnitName'].str.contains('Expansion')].drop_duplicates(subset=['TerminalName'], keep='first')
terms_df_onshore_noexpansions = terms_df_onshore.loc[~terms_df_onshore['UnitName'].str.contains('Expansion')].drop_duplicates(subset=['TerminalName'], keep='first')


In [136]:
onshore_export_regional_costs = pandas.DataFrame(index=region_list)

onshore_export_regional_costs['NumberOfDataPoints'] = terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Export')].groupby('Region')[['CostUSDPerMtpa']].count()

onshore_export_regional_costs['RegionalCostUSDPerMtpa'] = terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Export')].groupby('Region')[['CostUSDPerMtpa']].mean()

onshore_export_regional_costs.loc[(onshore_export_regional_costs.NumberOfDataPoints<3) | 
                                  (onshore_export_regional_costs.NumberOfDataPoints.isnull()), 
                                  'RegionalCostUSDPerMtpa'] = \
terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Export')][['CostUSDPerMtpa']].mean().values[0]
print('export onshore')
onshore_export_regional_costs['RegionalCostUSDPerMtpa'] = onshore_export_regional_costs['RegionalCostUSDPerMtpa']
onshore_export_regional_costs

export onshore


,NumberOfDataPoints,RegionalCostUSDPerMtpa
Australia and New Zealand,4.0,5.949954e+08
East Asia,NaN,5.892177e+08
Eurasia,1.0,5.892177e+08
Europe,NaN,5.892177e+08
Latin America and the Caribbean,3.0,5.239181e+08
Middle East and North Africa,6.0,5.447671e+08
North America,12.0,5.779734e+08
SE Asia,1.0,5.892177e+08
South Asia,NaN,5.892177e+08
Sub-Saharan Africa,5.0,6.235952e+08


In [137]:
onshore_import_regional_costs = pandas.DataFrame(index=region_list)

onshore_import_regional_costs['NumberOfDataPoints'] = terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Import')].groupby('Region')[['CostUSDPerMtpa']].count()

onshore_import_regional_costs['RegionalCostUSDPerMtpa'] = terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Import')].groupby('Region')[['CostUSDPerMtpa']].mean()

onshore_import_regional_costs.loc[(onshore_import_regional_costs.NumberOfDataPoints<3) | 
                                  (onshore_import_regional_costs.NumberOfDataPoints.isnull()), 
                                  'RegionalCostUSDPerMtpa'] = \
terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Import')][['CostUSDPerMtpa']].mean().values[0]
print('import onshore')
onshore_import_regional_costs['RegionalCostUSDPerMtpa'] = onshore_import_regional_costs['RegionalCostUSDPerMtpa']
onshore_import_regional_costs

import onshore


,NumberOfDataPoints,RegionalCostUSDPerMtpa
Australia and New Zealand,1.0,2.696893e+08
East Asia,15.0,3.516386e+08
Eurasia,NaN,2.696893e+08
Europe,15.0,2.325106e+08
Latin America and the Caribbean,4.0,3.104855e+08
Middle East and North Africa,2.0,2.696893e+08
North America,2.0,2.696893e+08
SE Asia,9.0,2.049370e+08
South Asia,7.0,2.102106e+08
Sub-Saharan Africa,NaN,2.696893e+08


In [138]:
floating_export_regional_costs = pandas.DataFrame(index=region_list)

floating_export_regional_costs['NumberOfDataPoints'] = terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Export')].groupby('Region')[['CostUSDPerMtpa']].count()

floating_export_regional_costs['RegionalCostUSDPerMtpa'] = terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Export')].groupby('Region')[['CostUSDPerMtpa']].mean()

floating_export_regional_costs.loc[:,
                                #(floating_export_regional_costs.NumberOfDataPoints<3) | 
                                #  (floating_export_regional_costs.NumberOfDataPoints.isnull()), 
                                  'RegionalCostUSDPerMtpa'] = \
terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Export')][['CostUSDPerMtpa']].mean().values[0]
print('export floating')
floating_export_regional_costs['RegionalCostUSDPerMtpa'] = floating_export_regional_costs['RegionalCostUSDPerMtpa']
floating_export_regional_costs

export floating


,NumberOfDataPoints,RegionalCostUSDPerMtpa
Australia and New Zealand,NaN,5.674729e+08
East Asia,NaN,5.674729e+08
Eurasia,1.0,5.674729e+08
Europe,NaN,5.674729e+08
Latin America and the Caribbean,NaN,5.674729e+08
Middle East and North Africa,NaN,5.674729e+08
North America,4.0,5.674729e+08
SE Asia,NaN,5.674729e+08
South Asia,NaN,5.674729e+08
Sub-Saharan Africa,4.0,5.674729e+08


In [139]:
floating_import_regional_costs = pandas.DataFrame(index=region_list)

floating_import_regional_costs['NumberOfDataPoints'] = terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Import')].groupby('Region')[['CostUSDPerMtpa']].count()

floating_import_regional_costs['RegionalCostUSDPerMtpa'] = terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Import')].groupby('Region')[['CostUSDPerMtpa']].mean()

floating_import_regional_costs.loc[:,
                                   #(floating_import_regional_costs.NumberOfDataPoints<3) | 
                                  #(floating_import_regional_costs.NumberOfDataPoints.isnull()), 
                                  'RegionalCostUSDPerMtpa'] = \
terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Import')][['CostUSDPerMtpa']].mean().values[0]
print('import floating')
floating_import_regional_costs['RegionalCostUSDPerMtpa'] = floating_import_regional_costs['RegionalCostUSDPerMtpa']
floating_import_regional_costs

import floating


,NumberOfDataPoints,RegionalCostUSDPerMtpa
Australia and New Zealand,3.0,1.346993e+08
East Asia,NaN,1.346993e+08
Eurasia,NaN,1.346993e+08
Europe,15.0,1.346993e+08
Latin America and the Caribbean,3.0,1.346993e+08
Middle East and North Africa,NaN,1.346993e+08
North America,NaN,1.346993e+08
SE Asia,4.0,1.346993e+08
South Asia,8.0,1.346993e+08
Sub-Saharan Africa,2.0,1.346993e+08


## global calculations

In [140]:
cost_onshore_export = terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Export')][['CostUSDPerMtpa']].mean().values
cost_onshore_export_std = terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Export')][['CostUSDPerMtpa']].std().values

cost_floating_export = terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Export')][['CostUSDPerMtpa']].mean().values
cost_floating_export_error = terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Export')][['CostUSDPerMtpa']].std().values

cost_onshore_import = terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Import')][['CostUSDPerMtpa']].mean().values
cost_onshore_import_error = terms_df_onshore_noexpansions.loc[(~terms_df_onshore_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_onshore_noexpansions['Import/Export']=='Import')][['CostUSDPerMtpa']].std().values

cost_floating_import = terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Import')][['CostUSDPerMtpa']].mean().values
cost_floating_import_error = terms_df_floating_noexpansions.loc[(~terms_df_floating_noexpansions.loc[:,'CostUSDPerMtpa'].isna())&
                                   (terms_df_floating_noexpansions['Import/Export']=='Import')][['CostUSDPerMtpa']].std().values

In [141]:
cost_onshore_export

array([5.89217718e+08])

In [142]:
cost_floating_export

array([5.67472856e+08])

In [143]:
cost_onshore_import

array([2.69689349e+08])

In [144]:
cost_floating_import

array([1.34699251e+08])

## calculate import terminal costs - AFTER invasion

In [145]:
terms_df_import2 = terms_df_orig2.loc[terms_df_orig2['Import/Export']=='Import'].reset_index(drop=True)

terms_df_import2['CostUSDTotal'] = numpy.nan

# for import
for region in region_list:
    # for floating
    terms_df_import2.loc[(terms_df_import2['Floating']=='yes')&(terms_df_import2['Region']==region),'CostUSDTotal'] = \
    terms_df_import2.loc[(terms_df_import2['Floating']=='yes')&(terms_df_import2['Region']==region),'CapacityInMtpa'] * \
    floating_import_regional_costs.loc[region,'RegionalCostUSDPerMtpa']
    # for onshore
    terms_df_import2.loc[(terms_df_import2['Floating']!='yes')&(terms_df_import2['Region']==region),'CostUSDTotal'] = \
    terms_df_import2.loc[(terms_df_import2['Floating']!='yes')&(terms_df_import2['Region']==region),'CapacityInMtpa'] * \
    onshore_import_regional_costs.loc[region,'RegionalCostUSDPerMtpa']   

# now replace the ones that we have exact numbers for...
costs_exist_TF = ~terms_df_import2['CostEstUSD'].isna()
terms_df_import2.loc[costs_exist_TF,'CostUSDTotal'] = \
    terms_df_import2[costs_exist_TF]['CostEstUSD']

## calculate export terminal costs - AFTER invasion

In [146]:
terms_df_export2 = terms_df_orig2.loc[terms_df_orig2['Import/Export']=='Export'].reset_index(drop=True)

terms_df_export2['CostUSDTotal'] = numpy.nan

# for import
for region in region_list:
    # for floating
    terms_df_export2.loc[(terms_df_export2['Floating']=='yes')&(terms_df_export2['Region']==region),'CostUSDTotal'] = \
    terms_df_export2.loc[(terms_df_export2['Floating']=='yes')&(terms_df_export2['Region']==region),'CapacityInMtpa'] * \
    floating_export_regional_costs.loc[region,'RegionalCostUSDPerMtpa']
    # for onshore
    terms_df_export2.loc[(terms_df_export2['Floating']!='yes')&(terms_df_export2['Region']==region),'CostUSDTotal'] = \
    terms_df_export2.loc[(terms_df_export2['Floating']!='yes')&(terms_df_export2['Region']==region),'CapacityInMtpa'] * \
    onshore_export_regional_costs.loc[region,'RegionalCostUSDPerMtpa']   

# now replace the ones that we have exact numbers for...
costs_exist_TF = ~terms_df_export2['CostEstUSD'].isna()
terms_df_export2.loc[costs_exist_TF,'CostUSDTotal'] = \
    terms_df_export2[costs_exist_TF]['CostEstUSD']

## now calculate regional/country level costs

### import terminals - AFTER invasion

In [147]:
#mtpa by country
cost_by_country_import_df2 = pandas.DataFrame(columns=status_list, index=country_list)
cost_by_region_import_df2 = pandas.DataFrame(columns=status_list, index=region_list)

for status in status_list:
    #print(status)
    cost_by_country_import_df2[status] = terms_df_import2.loc[terms_df_import2.Status==status].groupby('Country')['CostUSDTotal'].sum(min_count=1)

for status in status_list:
    #print(status)
    cost_by_region_import_df2[status] = terms_df_import2.loc[terms_df_import2.Status==status].groupby('Region')['CostUSDTotal'].sum(min_count=1)

cost_by_country_import_df2 = cost_by_country_import_df2.fillna(0)
cost_by_region_import_df2 = cost_by_region_import_df2.fillna(0)

# add total and proposed+construction info
cost_by_region_import_df2['Proposed+Construction'] = cost_by_region_import_df2[['Proposed','Construction']].sum(axis=1)
cost_by_region_import_df2.sort_values(by='Proposed+Construction', inplace=True)
cost_by_region_import_df2 = cost_by_region_import_df2[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
cost_by_region_import_df2.sort_values('Proposed+Construction', ascending=False, inplace=True)

cost_by_country_import_df2['Proposed+Construction'] = cost_by_country_import_df2[['Proposed','Construction']].sum(axis=1)
cost_by_country_import_df2.sort_values(by='Proposed+Construction', inplace=True)
cost_by_country_import_df2 = cost_by_country_import_df2[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
cost_by_country_import_df2.sort_values('Proposed+Construction', ascending=False, inplace=True)

# total
if 'Total' in cost_by_region_import_df2.index:
    cost_by_region_import_df2.drop(index='Total', inplace=True)   
    total = cost_by_region_import_df2.sum(axis=0)
    total.name='Total'
    cost_by_region_import_df2 = cost_by_region_import_df2.append(total)

else:
    total = cost_by_region_import_df2.sum(axis=0)
    total.name='Total'
    cost_by_region_import_df2 = cost_by_region_import_df2.append(total)

# total
if 'Total' in cost_by_country_import_df2.index:
    cost_by_country_import_df2.drop(index='Total', inplace=True)   
    total = cost_by_country_import_df2.sum(axis=0)
    total.name='Total'
    cost_by_country_import_df2 = cost_by_country_import_df2.append(total)

else:
    total = cost_by_country_import_df2.sum(axis=0)
    total.name='Total'
    cost_by_country_import_df2 = cost_by_country_import_df2.append(total)

# save countries
cost_by_country_import_df2 = cost_by_country_import_df2.loc[~(cost_by_country_import_df2==0).all(axis=1)]
cost_by_country_import_df2.to_excel('cost-by-country-import.xlsx')

cost_by_region_import_df2/1e9

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_10895/878003005.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_by_region_import_df2 = cost_by_region_import_df2.append(total)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_10895/878003005.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_by_country_import_df2 = cost_by_country_import_df2.append(total)


,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
East Asia,49.999150,31.296423,81.295573,7.127834,18.509968,154.528032,0.000000,0.527458,0.000000
Europe,25.896968,3.311242,29.208209,3.129799,18.624343,40.023233,0.000000,0.000000,0.404098
SE Asia,18.613397,2.830531,21.443929,2.903390,7.505797,7.209944,0.000000,0.000000,0.000000
South Asia,10.806167,5.689783,16.495950,1.763693,13.623280,12.273269,0.000000,0.000000,0.000000
Latin America and the Caribbean,4.550742,1.654107,6.204849,0.870157,4.374250,8.683066,0.000000,0.000000,0.000000
Middle East and North Africa,4.500000,0.000000,4.500000,1.495162,5.027009,8.469662,1.589451,0.000000,0.000000
Sub-Saharan Africa,3.170584,0.350000,3.520584,0.067350,1.037184,0.014817,0.000000,0.000000,0.000000
Australia and New Zealand,0.716910,0.172500,0.889410,0.000000,0.172500,0.000000,0.000000,0.000000,0.000000
North America,0.013484,0.000000,0.013484,0.000000,64.668235,20.456647,6.337263,12.136021,4.665626
Eurasia,0.000000,0.000000,0.000000,0.000000,0.000000,2.925737,0.000000,0.000000,0.000000


### export terminals - AFTER invasion

In [148]:
#mtpa by country
cost_by_country_export_df2 = pandas.DataFrame(columns=status_list, index=country_list)
cost_by_region_export_df2 = pandas.DataFrame(columns=status_list, index=region_list)

for status in status_list:
    #print(status)
    cost_by_country_export_df2[status] = terms_df_export2.loc[terms_df_export2.Status==status].groupby('Country')['CostUSDTotal'].sum(min_count=1)

for status in status_list:
    #print(status)
    cost_by_region_export_df2[status] = terms_df_export2.loc[terms_df_export2.Status==status].groupby('Region')['CostUSDTotal'].sum(min_count=1)

cost_by_country_export_df2 = cost_by_country_export_df2.fillna(0)
cost_by_region_export_df2 = cost_by_region_export_df2.fillna(0)

# add total and proposed+construction info
cost_by_region_export_df2['Proposed+Construction'] = cost_by_region_export_df2[['Proposed','Construction']].sum(axis=1)
cost_by_region_export_df2.sort_values(by='Proposed+Construction', inplace=True)
cost_by_region_export_df2 = cost_by_region_export_df2[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
cost_by_region_export_df2.sort_values('Proposed+Construction', ascending=False, inplace=True)

cost_by_country_export_df2['Proposed+Construction'] = cost_by_country_export_df2[['Proposed','Construction']].sum(axis=1)
cost_by_country_export_df2.sort_values(by='Proposed+Construction', inplace=True)
cost_by_country_export_df2 = cost_by_country_export_df2[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
cost_by_country_export_df2.sort_values('Proposed+Construction', ascending=False, inplace=True)

# total
if 'Total' in cost_by_region_export_df2.index:
    cost_by_region_export_df2.drop(index='Total', inplace=True)   
    total = cost_by_region_export_df2.sum(axis=0)
    total.name='Total'
    cost_by_region_export_df2 = cost_by_region_export_df2.append(total)

else:
    total = cost_by_region_export_df2.sum(axis=0)
    total.name='Total'
    cost_by_region_export_df2 = cost_by_region_export_df2.append(total)

# total
if 'Total' in cost_by_country_export_df2.index:
    cost_by_country_export_df2.drop(index='Total', inplace=True)   
    total = cost_by_country_export_df2.sum(axis=0)
    total.name='Total'
    cost_by_country_export_df2 = cost_by_country_export_df2.append(total)

else:
    total = cost_by_country_export_df2.sum(axis=0)
    total.name='Total'
    cost_by_country_export_df2 = cost_by_country_export_df2.append(total)

# save countries
cost_by_country_export_df2 = cost_by_country_export_df2.loc[~(cost_by_country_export_df2==0).all(axis=1)]
#cost_by_country_export_df2.to_excel('cost-by-country-export.xlsx')

cost_by_region_export_df2/1e9

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_10895/3290476363.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_by_region_export_df2 = cost_by_region_export_df2.append(total)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_10895/3290476363.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_by_country_export_df2 = cost_by_country_export_df2.append(total)


,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
North America,291.079718,30.201867,321.281585,11.164123,255.674029,42.712233,8.842993,0.866960,0.000000
Sub-Saharan Africa,87.060503,8.001305,95.061808,24.680967,17.401679,33.993814,0.000000,0.000000,0.000000
Eurasia,72.223061,19.100000,91.323061,14.918846,11.678511,18.030062,0.000000,0.000000,0.000000
Middle East and North Africa,54.317129,0.582901,54.900029,11.984877,58.593035,68.074428,0.000000,5.665578,4.249184
SE Asia,36.551966,2.378435,38.930402,0.883827,12.031095,55.565653,0.000000,0.000000,17.735453
Australia and New Zealand,28.301483,0.000000,28.301483,17.250000,30.920755,176.846423,0.000000,0.000000,0.000000
Latin America and the Caribbean,10.000000,0.000000,10.000000,0.000000,8.740743,6.481188,0.000000,0.974175,0.000000
East Asia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Europe,0.000000,0.000000,0.000000,0.000000,6.000000,6.182825,0.000000,0.000000,0.000000
South Asia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## calculate import terminal costs - BEFORE invasion

In [149]:
terms_df_import1 = terms_df_orig1.loc[terms_df_orig1['Import/Export']=='Import'].reset_index(drop=True)

terms_df_import1['CostUSDTotal'] = numpy.nan

# for import
for region in region_list:
    # for floating
    terms_df_import1.loc[(terms_df_import1['Floating']=='yes')&(terms_df_import1['Region']==region),'CostUSDTotal'] = \
    terms_df_import1.loc[(terms_df_import1['Floating']=='yes')&(terms_df_import1['Region']==region),'CapacityInMtpa'] * \
    floating_import_regional_costs.loc[region,'RegionalCostUSDPerMtpa']
    # for onshore
    terms_df_import1.loc[(terms_df_import1['Floating']!='yes')&(terms_df_import1['Region']==region),'CostUSDTotal'] = \
    terms_df_import1.loc[(terms_df_import1['Floating']!='yes')&(terms_df_import1['Region']==region),'CapacityInMtpa'] * \
    onshore_import_regional_costs.loc[region,'RegionalCostUSDPerMtpa']   

# now replace the ones that we have exact numbers for...
costs_exist_TF = ~terms_df_import1['CostEstUSD'].isna()
terms_df_import1.loc[costs_exist_TF,'CostUSDTotal'] = \
    terms_df_import1[costs_exist_TF]['CostEstUSD']

## calculate export terminal costs - BEFORE invasion

In [150]:
terms_df_export1 = terms_df_orig1.loc[terms_df_orig1['Import/Export']=='Export'].reset_index(drop=True)

terms_df_export1['CostUSDTotal'] = numpy.nan

# for import
for region in region_list:
    # for floating
    terms_df_export1.loc[(terms_df_export1['Floating']=='yes')&(terms_df_export1['Region']==region),'CostUSDTotal'] = \
    terms_df_export1.loc[(terms_df_export1['Floating']=='yes')&(terms_df_export1['Region']==region),'CapacityInMtpa'] * \
    floating_export_regional_costs.loc[region,'RegionalCostUSDPerMtpa']
    # for onshore
    terms_df_export1.loc[(terms_df_export1['Floating']!='yes')&(terms_df_export1['Region']==region),'CostUSDTotal'] = \
    terms_df_export1.loc[(terms_df_export1['Floating']!='yes')&(terms_df_export1['Region']==region),'CapacityInMtpa'] * \
    onshore_export_regional_costs.loc[region,'RegionalCostUSDPerMtpa']   

# now replace the ones that we have exact numbers for...
costs_exist_TF = ~terms_df_export1['CostEstUSD'].isna()
terms_df_export1.loc[costs_exist_TF,'CostUSDTotal'] = \
    terms_df_export1[costs_exist_TF]['CostEstUSD']

## now calculate regional/country level costs

### import terminals - BEFORE invasion

In [151]:
#mtpa by country
cost_by_country_import_df1 = pandas.DataFrame(columns=status_list, index=country_list)
cost_by_region_import_df1 = pandas.DataFrame(columns=status_list, index=region_list)

for status in status_list:
    #print(status)
    cost_by_country_import_df1[status] = terms_df_import1.loc[terms_df_import1.Status==status].groupby('Country')['CostUSDTotal'].sum(min_count=1)

for status in status_list:
    #print(status)
    cost_by_region_import_df1[status] = terms_df_import1.loc[terms_df_import1.Status==status].groupby('Region')['CostUSDTotal'].sum(min_count=1)

cost_by_country_import_df1 = cost_by_country_import_df1.fillna(0)
cost_by_region_import_df1 = cost_by_region_import_df1.fillna(0)

# add total and proposed+construction info
cost_by_region_import_df1['Proposed+Construction'] = cost_by_region_import_df1[['Proposed','Construction']].sum(axis=1)
cost_by_region_import_df1.sort_values(by='Proposed+Construction', inplace=True)
cost_by_region_import_df1 = cost_by_region_import_df1[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
cost_by_region_import_df1.sort_values('Proposed+Construction', ascending=False, inplace=True)

cost_by_country_import_df1['Proposed+Construction'] = cost_by_country_import_df1[['Proposed','Construction']].sum(axis=1)
cost_by_country_import_df1.sort_values(by='Proposed+Construction', inplace=True)
cost_by_country_import_df1 = cost_by_country_import_df1[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
cost_by_country_import_df1.sort_values('Proposed+Construction', ascending=False, inplace=True)

# total
if 'Total' in cost_by_region_import_df1.index:
    cost_by_region_import_df1.drop(index='Total', inplace=True)   
    total = cost_by_region_import_df1.sum(axis=0)
    total.name='Total'
    cost_by_region_import_df1 = cost_by_region_import_df1.append(total)

else:
    total = cost_by_region_import_df1.sum(axis=0)
    total.name='Total'
    cost_by_region_import_df1 = cost_by_region_import_df1.append(total)

# total
if 'Total' in cost_by_country_import_df1.index:
    cost_by_country_import_df1.drop(index='Total', inplace=True)   
    total = cost_by_country_import_df1.sum(axis=0)
    total.name='Total'
    cost_by_country_import_df1 = cost_by_country_import_df1.append(total)

else:
    total = cost_by_country_import_df1.sum(axis=0)
    total.name='Total'
    cost_by_country_import_df1 = cost_by_country_import_df1.append(total)

# save countries
cost_by_country_import_df1 = cost_by_country_import_df1.loc[~(cost_by_country_import_df1==0).all(axis=1)]
cost_by_country_import_df1.to_excel('cost-by-country-import.xlsx')

cost_by_region_import_df1/1e9

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_10895/2414572742.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_by_region_import_df1 = cost_by_region_import_df1.append(total)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_10895/2414572742.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_by_country_import_df1 = cost_by_country_import_df1.append(total)


,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
East Asia,48.902994,29.081780,77.984774,7.173427,8.491785,150.020524,0.000000,0.000000,0.000000
SE Asia,14.810080,4.935832,19.745912,0.351565,3.709731,8.328793,0.000000,0.000000,0.000000
South Asia,10.024627,8.145990,18.170617,3.551151,9.190507,12.975326,0.000000,0.000000,0.000000
Europe,12.160432,1.683836,13.844268,3.494591,19.750324,39.749635,1.255557,0.000000,0.404098
Middle East and North Africa,2.412614,5.916984,8.329599,0.471447,2.157515,5.982624,1.589451,0.000000,0.000000
Latin America and the Caribbean,5.925561,1.153026,7.078587,1.686314,3.194405,8.362091,0.808196,0.000000,0.000000
Sub-Saharan Africa,0.673496,0.228989,0.902485,0.501622,0.269399,0.000000,0.000000,0.000000,0.000000
North America,0.809068,0.000000,0.809068,1.375416,63.657639,25.283217,0.404098,14.509287,4.665626
Australia and New Zealand,0.700436,0.000000,0.700436,0.000000,0.471956,0.000000,0.000000,0.000000,0.000000
Eurasia,0.000000,0.000000,0.000000,0.000000,0.000000,2.925737,0.000000,0.000000,0.000000


### export terminals - BEFORE invasion

In [152]:
#mtpa by country
cost_by_country_export_df1 = pandas.DataFrame(columns=status_list, index=country_list)
cost_by_region_export_df1 = pandas.DataFrame(columns=status_list, index=region_list)

for status in status_list:
    #print(status)
    cost_by_country_export_df1[status] = terms_df_export1.loc[terms_df_export1.Status==status].groupby('Country')['CostUSDTotal'].sum(min_count=1)

for status in status_list:
    #print(status)
    cost_by_region_export_df1[status] = terms_df_export1.loc[terms_df_export1.Status==status].groupby('Region')['CostUSDTotal'].sum(min_count=1)

cost_by_country_export_df1 = cost_by_country_export_df1.fillna(0)
cost_by_region_export_df1 = cost_by_region_export_df1.fillna(0)

# add total and proposed+construction info
cost_by_region_export_df1['Proposed+Construction'] = cost_by_region_export_df1[['Proposed','Construction']].sum(axis=1)
cost_by_region_export_df1.sort_values(by='Proposed+Construction', inplace=True)
cost_by_region_export_df1 = cost_by_region_export_df1[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
cost_by_region_export_df1.sort_values('Proposed+Construction', ascending=False, inplace=True)

cost_by_country_export_df1['Proposed+Construction'] = cost_by_country_export_df1[['Proposed','Construction']].sum(axis=1)
cost_by_country_export_df1.sort_values(by='Proposed+Construction', inplace=True)
cost_by_country_export_df1 = cost_by_country_export_df1[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
cost_by_country_export_df1.sort_values('Proposed+Construction', ascending=False, inplace=True)

# total
if 'Total' in cost_by_region_export_df1.index:
    cost_by_region_export_df1.drop(index='Total', inplace=True)   
    total = cost_by_region_export_df1.sum(axis=0)
    total.name='Total'
    cost_by_region_export_df1 = cost_by_region_export_df1.append(total)

else:
    total = cost_by_region_export_df1.sum(axis=0)
    total.name='Total'
    cost_by_region_export_df1 = cost_by_region_export_df1.append(total)

# total
if 'Total' in cost_by_country_export_df1.index:
    cost_by_country_export_df1.drop(index='Total', inplace=True)   
    total = cost_by_country_export_df1.sum(axis=0)
    total.name='Total'
    cost_by_country_export_df1 = cost_by_country_export_df1.append(total)

else:
    total = cost_by_country_export_df1.sum(axis=0)
    total.name='Total'
    cost_by_country_export_df1 = cost_by_country_export_df1.append(total)

# save countries
cost_by_country_export_df1 = cost_by_country_export_df1.loc[~(cost_by_country_export_df1==0).all(axis=1)]
#cost_by_country_export_df1.to_excel('cost-by-country-export.xlsx')

cost_by_region_export_df1/1e9

/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_10895/3503850704.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_by_region_export_df1 = cost_by_region_export_df1.append(total)
/var/folders/h5/nfk59_vx7gg_58myc9jfk7_40000gn/T/ipykernel_10895/3503850704.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_by_country_export_df1 = cost_by_country_export_df1.append(total)


,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
North America,224.254927,28.956467,253.211394,15.969405,246.594463,44.856514,0.000000,0.866960,0.000000
Eurasia,15.272523,38.269470,53.541994,0.715016,8.897188,17.146236,0.000000,0.000000,0.000000
Sub-Saharan Africa,31.923851,10.300817,42.224668,83.234595,24.881450,20.755747,0.000000,1.995505,0.000000
Middle East and North Africa,28.986186,11.984877,40.971062,23.969753,25.744142,74.600409,0.000000,3.922323,4.249184
SE Asia,10.988560,3.384845,14.373406,0.883827,7.185932,38.361440,0.000000,0.000000,17.735453
Australia and New Zealand,13.892871,0.000000,13.892871,9.055591,20.600203,51.777562,0.000000,0.000000,0.000000
Latin America and the Caribbean,2.619591,0.000000,2.619591,0.000000,8.836416,10.347383,0.000000,0.255363,0.000000
East Asia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Europe,0.000000,0.000000,0.000000,0.000000,2.969657,0.265148,2.474714,0.000000,0.000000
South Asia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# compare Europe, Germany, globe

## export

In [153]:
cost_by_region_export_df1.loc['Total','Proposed+Construction']/1e9

420.834985581188

In [154]:
cost_by_region_export_df1.loc['Europe','Proposed+Construction']/1e9

0.0

GERMANY IS 0 FOR EXPORT

    #cost_by_country_export_df1.loc['Germany','Proposed+Construction']

## import

### before - in dev

In [161]:
for i in ['Germany','Europe','Total']:
    if i=='Germany':
        print('Germany')
        print(cost_by_country_import_df1.loc['Germany','Proposed+Construction']/1e9, 'billion USD cost')
        print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['TerminalID'].unique().size, 'terminals')
        
        print()
    if i=='Europe':
        print('Europe')
        print(cost_by_region_import_df1.loc['Europe','Proposed+Construction']/1e9, 'billion USD cost')
        print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['TerminalID'].unique().size, 'terminals')
        
        print()
    if i=='Europe':
        print('Global')
        print(cost_by_region_import_df1.loc['Total','Proposed+Construction']/1e9, 'billion USD cost')
        print(terms_df_orig1.loc[
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig1.loc[
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig1.loc[
                                 (terms_df_orig1.Status.isin(['Proposed','Construction']))]['TerminalID'].unique().size, 'terminals')

Germany
1.08414 billion USD cost
13.799999999999999 mtpa
2 projects
2 terminals

Europe
13.84426837844409 billion USD cost
81.17999999999999 mtpa
33 projects
28 terminals

Global
147.56574596359008 billion USD cost
1304.63 mtpa
420 projects
296 terminals


### before - operating

In [159]:
for i in ['Germany','Europe','Total']:
    if i=='Germany':
        print('Germany')
        print(cost_by_country_import_df1.loc['Germany','Operating']/1e9, 'billion USD cost')
        print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                                 (terms_df_orig1.Status.isin(['Operating']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                                 (terms_df_orig1.Status.isin(['Operating']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                                 (terms_df_orig1.Status.isin(['Operating']))]['TerminalID'].unique().size, 'terminals')
        
        print()
    if i=='Europe':
        print('Europe')
        print(cost_by_region_import_df1.loc['Europe','Operating']/1e9, 'billion USD cost')
        print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                                 (terms_df_orig1.Status.isin(['Operating']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                                 (terms_df_orig1.Status.isin(['Operating']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                                 (terms_df_orig1.Status.isin(['Operating']))]['TerminalID'].unique().size, 'terminals')
        
        print()
    if i=='Europe':
        print('Global')
        print(cost_by_region_import_df1.loc['Total','Operating']/1e9, 'billion USD cost')
        print(terms_df_orig1.loc[
                                 (terms_df_orig1.Status.isin(['Operating']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig1.loc[
                                 (terms_df_orig1.Status.isin(['Operating']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig1.loc[
                                 (terms_df_orig1.Status.isin(['Operating']))]['TerminalID'].unique().size, 'terminals')

Germany
0.0 billion USD cost
0.0 mtpa
0 projects
0 terminals

Europe
39.74963467476051 billion USD cost
178.77 mtpa
42 projects
41 terminals

Global
253.6279458857063 billion USD cost
1368.4299999999998 mtpa
321 projects
226 terminals


## import

### after - in dev

In [160]:
for i in ['Germany','Europe','Total']:
    if i=='Germany':
        print('Germany')
        print(cost_by_country_import_df2.loc['Germany','Proposed+Construction']/1e9, 'billion USD cost')
        print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['TerminalID'].unique().size, 'terminals')
        
        print()
    if i=='Europe':
        print('Europe')
        print(cost_by_region_import_df2.loc['Europe','Proposed+Construction']/1e9, 'billion USD cost')
        print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['TerminalID'].unique().size, 'terminals')
        
        print()
    if i=='Europe':
        print('Global')
        print(cost_by_region_import_df2.loc['Total','Proposed+Construction']/1e9, 'billion USD cost')
        print(terms_df_orig2.loc[
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig2.loc[
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['TerminalID'].count(), 'projects')
        print(terms_df_orig2.loc[
                                 (terms_df_orig2.Status.isin(['Proposed','Construction']))]['TerminalID'].unique().size, 'terminals')

Germany
8.547707408659813 billion USD cost
48.739999999999995 mtpa
8 projects
8 terminals

Europe
29.20820945659853 billion USD cost
165.32999999999998 mtpa
46 projects
41 terminals

Global
163.5719880903709 billion USD cost
1481.18 mtpa
431 projects
299 terminals


### after - operating

In [104]:
for i in ['Germany','Europe','Total']:
    if i=='Germany':
        print('Germany')
        print(cost_by_country_import_df2.loc['Germany','Operating']/1e9, 'billion USD cost')
        print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                                 (terms_df_orig2.Status.isin(['Operating']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                                 (terms_df_orig2.Status.isin(['Operating']))]['TerminalName'].count(), 'projects')
        print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                                 (terms_df_orig2.Status.isin(['Operating']))]['TerminalName'].unique().size, 'terminals')
        
        print()
    if i=='Europe':
        print('Europe')
        print(cost_by_region_import_df2.loc['Europe','Operating']/1e9, 'billion USD cost')
        print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                                 (terms_df_orig2.Status.isin(['Operating']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                                 (terms_df_orig2.Status.isin(['Operating']))]['TerminalName'].count(), 'projects')
        print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                                 (terms_df_orig2.Status.isin(['Operating']))]['TerminalName'].unique().size, 'terminals')
        
        print()
    if i=='Europe':
        print('Global')
        print(cost_by_region_import_df2.loc['Total','Operating']/1e9, 'billion USD cost')
        print(terms_df_orig2.loc[
                                 (terms_df_orig2.Status.isin(['Operating']))]['CapacityInMtpa'].sum(), 'mtpa')
        print(terms_df_orig2.loc[
                                 (terms_df_orig2.Status.isin(['Operating']))]['TerminalName'].count(), 'projects')
        print(terms_df_orig2.loc[
                                 (terms_df_orig2.Status.isin(['Operating']))]['TerminalName'].unique().size, 'terminals')

Germany
0.0 billion USD cost
0 mtpa
0 projects
0 terminals

Europe
40.037090926295924 billion USD cost
185.70999999999995 mtpa
45 projects
43 terminals

Global
256.59745216277673 billion USD cost
1382.1899999999994 mtpa
345 projects
233 terminals
